# Parsl Basics

<table style="width:90%">
    <tr><td style="text-align:left;"><a href="./1 - Welcome.ipynb">Welcome</a></td><td style="text-align:right;"><a href="./3 - Configurations.ipynb">Next (Configurations)</a></td></tr>
</table>

## Apps

* Parsl apps are the base unit of work.
* There are two flavors
  * Python app
  * Bash app
* They are really just Python functions with a decorator

### Python Apps
```python
@python_app
def random_multiply(x):
    import random
    return x * random.random()
```
Some Caveats
* Python apps are executed asynchronously, and potentially remotely
  * Resources cannot be assumed to be shared (e.g. globals, imports)
* Any Python object (primitives, files, classes, etc.) can be passed as input arguments
  * Must be serializable
* Files should be encapsulated in a Parsl File object
  * This make sure Parsl can make the file available to the function and track interdependencies
  * Files should be given as arguments to the function
  * We will discuss file more soon...
* stdout/err from remote machines may not be captured


In [2]:
import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config

parsl.load(config)

@python_app
def random_multiply(x):
    import random
    return x * random.random()

In [3]:
v = 5
print(f"Random multiply of {v} is {random_multiply(v)}")

In [4]:
v = 7
print(f"Random multiply of {v} is {random_multiply(v).result()}")

### Bash Apps
```python
@bash_app
def echo(word, stderr='std.err', stdout='std.out'):
    return f'echo "{word}"'
```
* The return of a `bash_app` should be a command that can be executed in a Bash environment
* No limit to the length of the command
* Files can still be passed as with Python apps
* stdout/err are captured and redirected to the given file names


In [5]:
@bash_app
def echo(word, stderr='std.err', stdout='std.out'):
    return f'echo "{word}"'

print(f"Return value is {echo('Hello world').result()}")
for line in open('std.out', 'r').readlines():
    print(line)



Note: Bash and Python apps return an `AppFuture` which is similar to `concurrent.futures.Future`. It is a promise that the wrapped code will be executed and the return results will be made available upon request.

### App Futures
* `done()` - non-blocking check on the status of the app (`True/False`)
* `running()` - Returns `True` if the task is currently executing
* `result([timeout])` - Returns the result of the executed function, blocks until the execution is complete unless a timeout is given
* `cancel()` - Cancel the future if possible


## Files

Parsl has a File object which abstracts access to a file regardless of where the code is executed. The File object can handle both local

```python
input_file = File("my.input.dat")    # wraps an existing file
output_file = File("my.output.dat")  # wraps a Future
```

and remote files.

```python
input_remote_file = File("https://raw.githubusercontent.com/Parsl/parsl-tutorial/master/input/unsorted.txt")  # wraps a remote file
```

Parsl supports Local, Globus, FTP, HTTP(S), and rsync access to remote files (the user is responsible for any credentials). This is done via `storage_access` arguments in the configuration.

In [6]:
from parsl.data_provider.files import File

# App that copies the contents of a file to another file
@bash_app
def replace(inputs=[], outputs=[]):
     return f"sed 's/1/!/g' {inputs[0]} > {outputs[0]}"

# Create Parsl file objects
input_remote_file = File("https://raw.githubusercontent.com/Parsl/parsl-tutorial/master/input/unsorted.txt")  # wraps a remote file
output_file = File("replaced.txt")

# Call the replace app with the remote file
repl_future = replace(inputs=[input_remote_file], outputs=[output_file])

# Read what was redirected to the output file
print(f"The file {output_file.filepath} contains:")
with open(repl_future.outputs[0].result(), 'r') as f:
     print(f.read())

<br><table style="width:90%">
    <tr><td style="text-align:left;"><a href="./1 - Welcome.ipynb">Welcome</a></td><td style="text-align:right;"><a href="./3 - Configurations.ipynb">Next (Configurations)</a></td></tr>
</table>